In [1]:
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
from pandasql import sqldf 
mysql = lambda q: sqldf(q, globals())

In [2]:
Inventory = pd.read_csv('Inventory.txt', sep="\t")
Inventory['depositDate'] = pd.to_datetime(Inventory['depositDate'])
Inventory[Inventory['suppliedAs']=='Exchange']

Transactions = pd.read_csv('Transactions_0207.txt', sep="\t")
Transactions['bookDate'] = pd.to_datetime(Transactions['bookDate'])
Transactions['cancelDate'] = pd.to_datetime(Transactions['cancelDate'])
Transactions['startDate'] = pd.to_datetime(Transactions['startDate'])
Transactions['endDate'] = pd.to_datetime(Transactions['endDate'])

Accounts = pd.read_csv("Account_0207.txt",sep = "\t")
Accounts['AccountStatus'].fillna('Active',inplace = True)
Accounts['JoinedDate'] = pd.to_datetime(Accounts['JoinedDate'])


/apps/cent7/jupyterhub/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (4,16,23,28,31) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
Credit = pd.read_csv('Credit balance.txt', sep="\t")
Credit['ExpiryDate'] = pd.to_datetime(Credit['ExpiryDate'], format='%d/%m/%Y %H:%M:%S', errors = 'coerce')
Credit = Credit[~Credit['accountID'].isin([-1,0])]

In [4]:
Events = pd.read_csv("Events.txt",sep = "\t")
Events['EventTimestamp'] = pd.to_datetime(Events['EventTimestamp'])

/apps/cent7/jupyterhub/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
Account_benefits = pd.read_csv("Account Benefits.txt",sep = "\t")
Account_benefits = Account_benefits.drop(columns = ['benefitGrp'])
Account_benefits['utcStartDate'] = pd.to_datetime(Account_benefits['utcStartDate'])
Account_benefits['utcEndDate'] = pd.to_datetime(Account_benefits['utcEndDate'])

In [6]:
AccountsExtraDetails = pd.read_csv("AccountsExtraDetails.txt",sep = "\t")

In [7]:
# Events[Events['paidType']!='Free'].head()
Events[Events['AccountID']==5190314].head()

,AccountID,EventTimestamp,eventKey,eventName,DepositID,ResortID,benefitRowKey,RequestID,paidType,benefitGrp,benefit
39152,5190314,2016-01-03 16:00:00,1,Joined Account,NaN,NaN,233584.0,NaN,Paid,DAE GA,ASA Gold Advantage
442466,5190314,2016-01-04 10:15:00,3,Purchased Something,20000000.0,NaN,233584.0,NaN,Paid,DAE GA,ASA Gold Advantage
2185318,5190314,2019-01-25 07:15:00,6,Member Search Availability,NaN,NaN,NaN,43285527.0,Free,DAE Free,ASA Free Member


## Dividing the transactions data
Pre Period : 01/10/2016 to 30/09/2019
\
Post Period : 01/10/2019 to 30/03/2020

In [8]:
Transactions.head(2)
# accountlevel[(accountlevel['lastpurchase_now_dormancy']<60 ) & (accountlevel['Purchased']==1)& (accountlevel['search_score']==0)].head()


,finRowId,accountId,txnDate,bookDate,cancelDate,startDate,endDate,depositid,txnStatus,totalAmount,amount,gstAmount,cancelAmount,cancelGstAmount,promoAmount,cancelPromoAmount,promoCode,currency,txnCount,finOfficeId,resortKey,unitType,unitTypeDesc,unitSleeps,description,activeBooking,ReasonCode,updatedDate,ExcludeCharge,chargeGroup,priceTierType,createuser,createusername
0,462805,1301,1997-09-14,1997-09-14,NaT,1997-11-15,1997-11-22,1404,B,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,AUD,1,AU,AUSBHOS,3.0,2 Bedroom,6,Converted Charge,1,NaN,2021-10-22 07:02:18.647,NO,Exchange,None,99,E.M.S Supervisor
1,462807,1301,1998-04-21,1998-04-21,NaT,1998-05-09,1998-05-16,2336,B,125.0,125.0,0.0,NaN,NaN,NaN,NaN,NaN,AUD,1,AU,AUSSHEW,2.0,1 Bedroom,4,Converted Charge,1,NaN,2021-10-22 07:02:18.647,NO,Bonus,None,99,E.M.S Supervisor


In [9]:
Pre_transaction = Transactions[(Transactions['bookDate']>='2016-10-01') & (Transactions['bookDate']<'2019-10-01')]
                
Post_transaction = Transactions[(Transactions['bookDate']>='2019-10-01') & (Transactions['bookDate']<='2020-03-30')]

USER LEVEL TABLE FROM PRE DATA

In [10]:
accountlevel = pd.DataFrame(Pre_transaction['accountId'].unique())
accountlevel.columns = ['accountId']

accountlevel.count()
# head(2)

accountId    68098
dtype: int64

In [11]:
accountlevel.head(2)

,accountId
0,71050
1,1137640


CREATING OUR LABEL OF PURCHASE = 1/0

In [12]:
next_purchase = Post_transaction.groupby('accountId').bookDate.min().reset_index()
next_purchase.columns = ['accountId','MinPurchaseDate_PostPeriod']
next_purchase['Purchased'] = 1

In [13]:
next_purchase.head(2)

,accountId,MinPurchaseDate_PostPeriod,Purchased
0,1341,2020-01-07,1
1,1359,2019-11-05,1


In [14]:
next_purchase.accountId.nunique()

17864

68,000 Unique customers purchased in last 3 years, 17000 unique customers purchased in the next 6 months

In [15]:
pre_purchase = Pre_transaction.groupby('accountId').bookDate.max().reset_index()
pre_purchase.columns = ['accountId','MaxPurchaseDate_PrePeriod']

In [16]:
pre_purchase.head(2)

,accountId,MaxPurchaseDate_PrePeriod
0,1303,2019-06-03
1,1323,2018-01-23


In [17]:
merge_purchase = pd.merge(pre_purchase,next_purchase,on='accountId',how='left')

In [18]:
merge_purchase.tail()

,accountId,MaxPurchaseDate_PrePeriod,MinPurchaseDate_PostPeriod,Purchased
68093,7080561,2019-09-24,NaT,NaN
68094,7080873,2019-09-24,NaT,NaN
68095,7081226,2019-09-27,NaT,NaN
68096,7081433,2019-09-30,2019-12-12,1.0
68097,7081434,2019-09-30,NaT,NaN


Out of 68K unique customers in Pre Period, 13K purchased again in post period (~20%)

In [19]:
merge_purchase['diff_purchase_days'] = (merge_purchase['MinPurchaseDate_PostPeriod'] - merge_purchase['MaxPurchaseDate_PrePeriod']).dt.days

In [20]:
accountlevel.count()

accountId    68098
dtype: int64

In [21]:
accountlevel = pd.merge(accountlevel, merge_purchase,on='accountId',how='left')


In [22]:
accountlevel.head(2)

,accountId,MaxPurchaseDate_PrePeriod,MinPurchaseDate_PostPeriod,Purchased,diff_purchase_days
0,71050,2019-05-21,2020-03-25,1.0,309.0
1,1137640,2019-09-03,2019-12-21,1.0,109.0


In [23]:
accountlevel = pd.merge(accountlevel,Accounts[['accountid', 'JoinedDate','BSPCode']], how='left', left_on='accountId', right_on='accountid')

In [24]:
accountlevel = accountlevel[['accountId','MaxPurchaseDate_PrePeriod','MinPurchaseDate_PostPeriod','Purchased','diff_purchase_days','JoinedDate','BSPCode']]

In [25]:
accountlevel['Purchased'] = accountlevel['Purchased'].fillna(0)

In [26]:
accountlevel.head(2)

,accountId,MaxPurchaseDate_PrePeriod,MinPurchaseDate_PostPeriod,Purchased,diff_purchase_days,JoinedDate,BSPCode
0,71050,2019-05-21,2020-03-25,1.0,309.0,2009-02-19 14:00:00,AUS
1,1137640,2019-09-03,2019-12-21,1.0,109.0,2014-07-28 14:00:00,AUS


DORMANCY PERIOD : LAST PURCHASE IN PRE PERIOD TO '2019-10-01'

In [27]:
accountlevel['lastpurchase_now_dormancy'] = (pd.to_datetime('2019-10-01') - accountlevel['MaxPurchaseDate_PrePeriod']).dt.days

Country of Customer

In [28]:
accountlevel = pd.merge(accountlevel,AccountsExtraDetails[['AccountID','country']], how='left', left_on='accountId',right_on='AccountID')
accountlevel['country'] = accountlevel['country'].fillna('Australia')

In [29]:
accountlevel = accountlevel.drop(columns=['BSPCode', 'AccountID'])

SEARCHES

In [30]:
Pre_event = Events[(Events['EventTimestamp']>='2016-10-01') & (Events['EventTimestamp']<'2019-10-01')]
Pre_event['month_year'] = pd.to_datetime(Pre_event['EventTimestamp']).dt.to_period('M')
Pre_search_event  = Pre_event[Pre_event['eventName']== 'Member Search Availability']
Pre_request_event = Pre_event[Pre_event['eventName']== 'Member Request Placed']

/tmp/ipykernel_184557/778386704.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Pre_event['month_year'] = pd.to_datetime(Pre_event['EventTimestamp']).dt.to_period('M')


In [80]:
# Pre_transaction['book_month_year'] = pd.to_datetime(Pre_transaction['bookDate']).dt.to_period('M')
# # Pre_transaction.head()
# transact_yr = Pre_transaction.groupby(['accountId','book_month_year']).finRowId.count().reset_index()
# transact_yr_pivot = transact_yr.pivot(index=['accountId'],
#               columns=['book_month_year'], values='finRowId').reset_index().rename_axis(None, axis=1)
# # transact_yr_pivot.columns
# transact_yr_pivot.columns = ['accountId',     'Trans_2016_10','Trans_2016_11','Trans_2016_12','Trans_2017_01',
# 'Trans_2017_02','Trans_2017_03','Trans_2017_04','Trans_2017_05','Trans_2017_06',
# 'Trans_2017_07','Trans_2017_08','Trans_2017_09','Trans_2017_10','Trans_2017_11',
# 'Trans_2017_12','Trans_2018_01','Trans_2018_02','Trans_2018_03','Trans_2018_04',
# 'Trans_2018_05','Trans_2018_06','Trans_2018_07','Trans_2018_08','Trans_2018_09',
# 'Trans_2018_10','Trans_2018_11','Trans_2018_12','Trans_2019_01','Trans_2019_02',
# 'Trans_2019_03','Trans_2019_04','Trans_2019_05','Trans_2019_06','Trans_2019_07',
# 'Trans_2019_08','Trans_2019_09']
# transact_yr_pivot = transact_yr_pivot.fillna(0)

/tmp/ipykernel_109908/2748556516.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Pre_transaction['book_month_year'] = pd.to_datetime(Pre_transaction['bookDate']).dt.to_period('M')


In [81]:
# transact_yr_pivot.head()

,accountId,Trans_2016_10,Trans_2016_11,Trans_2016_12,Trans_2017_01,Trans_2017_02,Trans_2017_03,Trans_2017_04,Trans_2017_05,Trans_2017_06,Trans_2017_07,Trans_2017_08,Trans_2017_09,Trans_2017_10,Trans_2017_11,Trans_2017_12,Trans_2018_01,Trans_2018_02,Trans_2018_03,Trans_2018_04,Trans_2018_05,Trans_2018_06,Trans_2018_07,Trans_2018_08,Trans_2018_09,Trans_2018_10,Trans_2018_11,Trans_2018_12,Trans_2019_01,Trans_2019_02,Trans_2019_03,Trans_2019_04,Trans_2019_05,Trans_2019_06,Trans_2019_07,Trans_2019_08,Trans_2019_09
0,1303,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,1323,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1328,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1335,0.0,0.0,5.0,1.0,0.0,0.0,1.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0


In [31]:
search_yr = Pre_search_event.groupby(['AccountID','month_year']).eventKey.count().reset_index()
search_yr_pivot = search_yr.pivot(index=['AccountID'],
              columns=['month_year'], values='eventKey').reset_index().rename_axis(None, axis=1)

# request_yr = Pre_request_event.groupby(['AccountID','month_year']).eventKey.count().reset_index()
# request_yr_pivot = request_yr.pivot(index=['AccountID'],
#               columns=['month_year'], values='eventKey').reset_index().rename_axis(None, axis=1)

In [32]:
colnames=[]

for i in search_yr_pivot.columns.tolist():
    colnames.append('search_'+str(i).replace('-','_'))

In [33]:
colnames

['search_AccountID',
 'search_2018_10',
 'search_2018_11',
 'search_2018_12',
 'search_2019_01',
 'search_2019_02',
 'search_2019_03',
 'search_2019_04',
 'search_2019_05']

In [34]:
# search_yr_pivot.columns =  ['accountId',     'search_2018_10',     'search_2018_11',     'search_2018_12',     'search_2019_01',
#            'search_2019_02',     'search_2019_03',     'search_2019_04',     'search_2019_05']
search_yr_pivot.columns = colnames


In [35]:
# request_yr_pivot.columns = ['accountId',
#                              'Year_2016_10','Year_2016_11',
#                              'Year_2016_12',
#                              'Year_2017_01',
# 'Year_2017_02','Year_2017_03','Year_2017_04','Year_2017_05','Year_2017_06',
# 'Year_2017_07','Year_2017_08','Year_2017_09','Year_2017_10','Year_2017_11',
# 'Year_2017_12','Year_2018_01','Year_2018_02','Year_2018_03','Year_2018_04',
# 'Year_2018_05','Year_2018_06','Year_2018_07','Year_2018_08','Year_2018_09',
# 'Year_2018_10','Year_2018_11','Year_2018_12','Year_2019_01','Year_2019_02',
# 'Year_2019_03','Year_2019_04','Year_2019_05','Year_2019_06','Year_2019_07',
# 'Year_2019_08','Year_2019_09'
# ]


In [36]:
search_yr_pivot = search_yr_pivot.fillna(0)
# request_yr_pivot = request_yr_pivot.fillna(0)


In [37]:
search_yr_pivot.head()

,search_AccountID,search_2018_10,search_2018_11,search_2018_12,search_2019_01,search_2019_02,search_2019_03,search_2019_04,search_2019_05
0,1303,0.0,0.0,0.0,0.0,0.0,0.0,20.0,0.0
1,1348,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0
2,1351,0.0,1.0,41.0,0.0,0.0,0.0,0.0,0.0
3,1356,3.0,0.0,0.0,3.0,0.0,2.0,0.0,0.0
4,1359,0.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0


In [38]:
accountlevel = pd.merge(accountlevel,search_yr_pivot, how='left', left_on='accountId',right_on='search_AccountID')
# accountlevel = pd.merge(accountlevel,request_yr_pivot, how='left', left_on='accountId',right_on='accountId')

In [39]:
accountlevel.head(3)

,accountId,MaxPurchaseDate_PrePeriod,MinPurchaseDate_PostPeriod,Purchased,diff_purchase_days,JoinedDate,lastpurchase_now_dormancy,country,search_AccountID,search_2018_10,search_2018_11,search_2018_12,search_2019_01,search_2019_02,search_2019_03,search_2019_04,search_2019_05
0,71050,2019-05-21,2020-03-25,1.0,309.0,2009-02-19 14:00:00,133,New Zealand,71050.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
1,1137640,2019-09-03,2019-12-21,1.0,109.0,2014-07-28 14:00:00,28,New Zealand,1137640.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,39580,2016-10-01,NaT,0.0,NaN,2004-08-10 14:00:00,1095,Australia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
accountlevel.iloc[:, 8:] =accountlevel.iloc[:, 8:].fillna(0)

accountlevel['total_searches'] = accountlevel.iloc[:, 9:16].sum(axis=1)
# accountlevel['total_request'] = accountlevel.iloc[:, 16:51].sum(axis=1)

In [41]:
accountlevel.head()

,accountId,MaxPurchaseDate_PrePeriod,MinPurchaseDate_PostPeriod,Purchased,diff_purchase_days,JoinedDate,lastpurchase_now_dormancy,country,search_AccountID,search_2018_10,search_2018_11,search_2018_12,search_2019_01,search_2019_02,search_2019_03,search_2019_04,search_2019_05,total_searches
0,71050,2019-05-21,2020-03-25,1.0,309.0,2009-02-19 14:00:00,133,New Zealand,71050.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0
1,1137640,2019-09-03,2019-12-21,1.0,109.0,2014-07-28 14:00:00,28,New Zealand,1137640.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
2,39580,2016-10-01,NaT,0.0,NaN,2004-08-10 14:00:00,1095,Australia,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,79090,2016-10-01,NaT,0.0,NaN,2009-08-19 05:07:07,1095,Australia,79090.0,0.0,0.0,2.0,0.0,8.0,0.0,5.0,0.0,15.0
4,1120025,2019-09-06,NaT,0.0,NaN,2013-10-14 02:28:09,25,Australia,1120025.0,0.0,0.0,0.0,1.0,7.0,0.0,0.0,0.0,8.0


In [42]:
accountlevel['search_score']=0
# accountlevel['request_score']=0

In [47]:
#learning rate loop
alpha = 0.2
alpha_1_minus = 1-alpha
lt=[]
for i in range(1,len(search_yr_pivot.columns)):
    lt.append(alpha*(1-alpha)**i)

# req_lt=[]
# for i in range(1,len(request_yr_pivot.columns)):
#     req_lt.append(alpha*(1-alpha)**i)
    
# transact_lt=[]
# for i in range(1,len(transact_yr_pivot.columns)):
#     transact_lt.append(alpha*(1-alpha)**i)

In [48]:
lt = lt[::-1].copy()
# req_lt = req_lt[::-1].copy()
# transact_lt = transact_lt[::-1].copy()

In [49]:
(lt)

[0.033554432000000016,
 0.041943040000000015,
 0.052428800000000025,
 0.06553600000000002,
 0.08192000000000002,
 0.10240000000000003,
 0.12800000000000003,
 0.16000000000000003]

In [50]:
#SEARCH SCORE
j=0
for i in range(9,9+len(lt)):
    accountlevel['search_score']=accountlevel['search_score']+accountlevel.iloc[:,i]*lt[j]
    j+=1

In [47]:
# #REQUEST SCORE
# j=0
# for i in range(16,16+len(req_lt)):
#     accountlevel['request_score']=accountlevel['request_score']+accountlevel.iloc[:,i]*req_lt[j]
#     j+=1

In [48]:
# #Transaction Score
# transact_yr_pivot['transact_score']=0
# j=0
# for i in range(1,1+len(transact_lt)):
#     transact_yr_pivot['transact_score']=transact_yr_pivot['transact_score']+transact_yr_pivot.iloc[:,i]*transact_lt[j]
#     j+=1

In [49]:
# accountlevel = pd.merge(accountlevel,transact_yr_pivot[['accountId','transact_score']], how='left', left_on='accountId',right_on='accountId')


In [52]:
accountlevel.head(5)

,accountId,MaxPurchaseDate_PrePeriod,MinPurchaseDate_PostPeriod,Purchased,diff_purchase_days,JoinedDate,lastpurchase_now_dormancy,country,search_AccountID,search_2018_10,search_2018_11,search_2018_12,search_2019_01,search_2019_02,search_2019_03,search_2019_04,search_2019_05,total_searches,search_score
0,71050,2019-05-21,2020-03-25,1.0,309.0,2009-02-19 14:00:00,133,New Zealand,71050.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,0.256000
1,1137640,2019-09-03,2019-12-21,1.0,109.0,2014-07-28 14:00:00,28,New Zealand,1137640.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.134218
2,39580,2016-10-01,NaT,0.0,NaN,2004-08-10 14:00:00,1095,Australia,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,79090,2016-10-01,NaT,0.0,NaN,2009-08-19 05:07:07,1095,Australia,79090.0,0.0,0.0,2.0,0.0,8.0,0.0,5.0,0.0,15.0,1.400218
4,1120025,2019-09-06,NaT,0.0,NaN,2013-10-14 02:28:09,25,Australia,1120025.0,0.0,0.0,0.0,1.0,7.0,0.0,0.0,0.0,8.0,0.638976


In [131]:
accountlevel.to_csv('final_clustering.csv')
accountlevel.head()

,accountId,MaxPurchaseDate_PrePeriod,MinPurchaseDate_PostPeriod,Purchased,diff_purchase_days,JoinedDate,lastpurchase_now_dormancy,country,search_AccountID,search_2018_10,search_2018_11,search_2018_12,search_2019_01,search_2019_02,search_2019_03,search_2019_04,search_2019_05,total_searches,search_score,request_score
0,71050,2019-05-21,2020-03-25,1.0,309.0,2009-02-19 14:00:00,133,New Zealand,71050.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,0.256000,0
1,1137640,2019-09-03,2019-12-21,1.0,109.0,2014-07-28 14:00:00,28,New Zealand,1137640.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.134218,0
2,39580,2016-10-01,NaT,0.0,NaN,2004-08-10 14:00:00,1095,Australia,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0
3,79090,2016-10-01,NaT,0.0,NaN,2009-08-19 05:07:07,1095,Australia,79090.0,0.0,0.0,2.0,0.0,8.0,0.0,5.0,0.0,15.0,1.400218,0
4,1120025,2019-09-06,NaT,0.0,NaN,2013-10-14 02:28:09,25,Australia,1120025.0,0.0,0.0,0.0,1.0,7.0,0.0,0.0,0.0,8.0,0.638976,0


In [156]:
min_sc = accountlevel['search_score'].min()
max_sc= accountlevel['search_score'].max()
accountlevel['norm_search_score'] = (accountlevel['search_score'] - min_sc)/(max_sc - min_sc)
high_search_score_cutoff = accountlevel['norm_search_score'].quantile(0.9)
accountlevel['Dormancy_mon'] = (accountlevel['lastpurchase_now_dormancy']/30).round(1)

In [157]:
accountlevel['High_Search_Flag'] = accountlevel['norm_search_score']>high_search_score_cutoff

In [158]:
accountlevel_0_6_months = accountlevel[accountlevel['Dormancy_mon']<6]
accountlevel_6_12months = accountlevel[(accountlevel['Dormancy_mon']>=6) & (accountlevel['Dormancy_mon']<12)]
accountlevel_12_24months = accountlevel[(accountlevel['Dormancy_mon']>=12) & (accountlevel['Dormancy_mon']<24)]
accountlevel_morethan_24months = accountlevel[(accountlevel['Dormancy_mon']>=24)]

In [162]:
# high_search_score_cutoff
accountlevel.to_csv('Final_Cluster_File.csv',index=False)

--------------------- THE END OF NOTEBOOK ------------------------------

## APPENDIX

In [119]:
accountlevel.sort_values(by='search_score',ascending=False).head()
#accountlevel.to_csv('cluster_search_request_dormancy1.csv')

,accountId,MaxPurchaseDate_PrePeriod,MinPurchaseDate_PostPeriod,Purchased,diff_purchase_days,JoinedDate,lastpurchase_now_dormancy,country,search_2018_10,search_2018_11,search_2018_12,search_2019_01,search_2019_02,search_2019_03,search_2019_04,search_2019_05,Year_2016_10,Year_2016_11,Year_2016_12,Year_2017_01,Year_2017_02,Year_2017_03,Year_2017_04,Year_2017_05,Year_2017_06,Year_2017_07,Year_2017_08,Year_2017_09,Year_2017_10,Year_2017_11,Year_2017_12,Year_2018_01,Year_2018_02,Year_2018_03,Year_2018_04,Year_2018_05,Year_2018_06,Year_2018_07,Year_2018_08,Year_2018_09,Year_2018_10,Year_2018_11,Year_2018_12,Year_2019_01,Year_2019_02,Year_2019_03,Year_2019_04,Year_2019_05,Year_2019_06,Year_2019_07,Year_2019_08,Year_2019_09,total_searches,search_score,request_score
37492,5150625,2019-09-11,NaT,0.0,NaN,2014-03-26 07:52:09,20,India,298.0,1557.0,1137.0,1559.0,2306.0,1929.0,1556.0,216.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,3.0,3.0,1.0,4.0,0.0,0.0,7.0,9.0,8.0,10.0,2.0,1.0,10616.0,815.890379,0.407371
34499,189696,2019-08-12,2020-02-28,1.0,200.0,2009-01-19 07:00:00,50,United States,112.0,426.0,181.0,720.0,763.0,584.0,518.0,31.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,4.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,5.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3353.0,253.353869,0.577860
2533,70229,2019-08-07,2019-11-29,1.0,114.0,2008-12-21 14:00:00,55,Australia,176.0,301.0,502.0,1114.0,214.0,122.0,140.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,1.0,0.0,1.0,0.0,2.0,0.0,3.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,4.0,0.0,1.0,1.0,1.0,1.0,3.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,2599.0,235.644723,0.336591
234,12266,2019-09-30,2019-10-03,1.0,3.0,2000-11-29 14:00:00,1,Australia,133.0,216.0,792.0,433.0,195.0,609.0,339.0,18.0,0.0,2.0,4.0,3.0,0.0,0.0,0.0,0.0,2.0,2.0,1.0,3.0,0.0,2.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,4.0,3.0,0.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0,3.0,2775.0,225.031490,1.110210
47974,5048933,2019-04-25,2019-10-11,1.0,169.0,2005-03-30 16:00:00,159,Singapore,73.0,862.0,436.0,98.0,157.0,102.0,611.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,5.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,2374.0,216.592181,0.010122


In [132]:
# accountlevel.head()
transact_yr_pivot[transact_yr_pivot['accountId']==39580]

,accountId,Trans_2016_10,Trans_2016_11,Trans_2016_12,Trans_2017_01,Trans_2017_02,Trans_2017_03,Trans_2017_04,Trans_2017_05,Trans_2017_06,Trans_2017_07,Trans_2017_08,Trans_2017_09,Trans_2017_10,Trans_2017_11,Trans_2017_12,Trans_2018_01,Trans_2018_02,Trans_2018_03,Trans_2018_04,Trans_2018_05,Trans_2018_06,Trans_2018_07,Trans_2018_08,Trans_2018_09,Trans_2018_10,Trans_2018_11,Trans_2018_12,Trans_2019_01,Trans_2019_02,Trans_2019_03,Trans_2019_04,Trans_2019_05,Trans_2019_06,Trans_2019_07,Trans_2019_08,Trans_2019_09
9364,39580,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


XXXXXXXXXXXXXXXXXXXX TRANSACTIONS XXXXXXXXXXXXXXXXXXXXXXXXXXXXX

Customers who have not bought before

In [101]:
search_new = search_yr_pivot[(search_yr_pivot['search_2018_10']==0) and (search_yr_pivot['search_2018_11']==0) and (search_yr_pivot['search_2018_12']==0) and (search_yr_pivot['search_2019_01']>0) and (search_yr_pivot['search_2019_02']>0) and (search_yr_pivot['search_2019_03']>0) and (search_yr_pivot['search_2019_04']>0) and (search_yr_pivot['search_2019_05']>0)]
# search_yr_pivot.columns

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [98]:
search_yr_pivot.columns
search_new = search_yr_pivot[['accountId',
       'search_2019_01', 'search_2019_02', 'search_2019_03', 'search_2019_04',
       'search_2019_05']]

In [99]:
search_new.head()

,accountId,search_2019_01,search_2019_02,search_2019_03,search_2019_04,search_2019_05
0,1303,0.0,0.0,0.0,20.0,0.0
1,1348,0.0,3.0,0.0,0.0,0.0
2,1351,0.0,0.0,0.0,0.0,0.0
3,1356,3.0,0.0,2.0,0.0,0.0
4,1359,0.0,0.0,16.0,0.0,0.0


In [84]:
Last_6months_transc = pd.DataFrame(Pre_transaction[(Pre_transaction['bookDate']>='2019-04-01') & (Pre_transaction['bookDate']<='2019-09-30')].accountId.unique())

In [85]:
Last_6months_transc.columns = ['Last6_months_acc']

In [86]:
table1 = pd.merge(search_new,Last_6months_transc,how='left', left_on='accountId',right_on='Last6_months_acc')

In [87]:
post_acc = pd.DataFrame(Post_transaction['accountId'].unique())
post_acc.columns = ['post_acc']
table1 = pd.merge(table1,post_acc,how='left', left_on='accountId', right_on='post_acc')

In [88]:
table1.head()

,accountId,search_2019_01,search_2019_02,search_2019_03,search_2019_04,search_2019_05,Last6_months_acc,post_acc
0,1303,0.0,0.0,0.0,20.0,0.0,1303.0,NaN
1,1348,0.0,3.0,0.0,0.0,0.0,1348.0,NaN
2,1351,0.0,0.0,0.0,0.0,0.0,NaN,NaN
3,1356,3.0,0.0,2.0,0.0,0.0,1356.0,NaN
4,1359,0.0,0.0,16.0,0.0,0.0,NaN,1359.0


In [90]:
table1 = table1[table1['Last6_months_acc'].isna()]

In [92]:
table1.to_csv('new_searcher.csv')